Classificação de Texto com spaCy

Etapa 1: importação e Instalação das Bibliotacas

In [ ]:
import pandas as pd
import string
import spacy
import random
import seaborn as sns
import numpy as np

In [ ]:
pln = spacy.load('pt_core_news_sm')
pln

Etapa 2: Carregamento da base de dados

In [ ]:
base_dados = pd.read_csv('base_treinamento.txt', encoding = 'utf-8')

In [ ]:
base_dados.shape

In [ ]:
base_dados.head()

In [ ]:
base_dados.tail()

In [ ]:
sns.countplot(base_dados['emocao'], label = 'Contagem');

Etapa 3: Função para pré-processamento dos textos

In [ ]:
pontuacoes = string.punctuation
pontuacoes

In [ ]:
from spacy.lang.pt.stop_words import STOP_WORDS
stop_words = STOP_WORDS

In [ ]:
print(stop_words)

In [ ]:
len(stop_words)

In [ ]:
pln = spacy.load('pt_core_news_sm')
pln

In [ ]:
def preprocessamento(texto):
    texto = texto.lower()
    documento = pln(texto)

    lista = []
    for token in documento:
        # lista.append(token.text)
        lista.append(token.lemma_)

    lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in pontuacoes]
    lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
    
    return lista

In [ ]:
teste = preprocessamento('Estou aPrendendo processamento de linguagem natural, Curso em Curitiba 1 10 23')
teste

Etapa 4: Pré-processamento da base de dados

In [ ]:
base_dados['texto'] = base_dados['texto'].apply(preprocessamento)

In [ ]:
base_dados.head(10)

Tratamento da classe

In [ ]:
exemplo_base_dados = [["este trabalho é agradável", {"ALEGRIA": True, "MEDO": True}],
                      ["este lugar continua assutador", {"ALEGRIA": True, "MEDO": True}]]

In [ ]:
type(exemplo_base_dados)

In [ ]:
exemplo_base_dados[0]

In [ ]:
exemplo_base_dados[0][0]

In [ ]:
exemplo_base_dados[0][1]

In [ ]:
type(exemplo_base_dados[0][1])

In [ ]:
base_dados_final = []
for texto, emocao in zip(base_dados['texto'], base_dados['emocao']):
    # print(texto, emocao)
    if emocao == 'alegria':
        dic = ({'ALEGRIA': True, 'MEDO': False})
    elif emocao == 'medo':
        dic = ({'ALEGRIA': False, 'MEDO': True})

    base_dados_final.append([texto, dic.copy()])

In [ ]:
len(base_dados_final)

In [ ]:
base_dados_final[0]

In [ ]:
base_dados_final[0][0]

In [ ]:
base_dados_final[0][1]

In [ ]:
type(base_dados_final[0][1])

In [ ]:
base_dados_final

Etapa 5: Criação do Classificador

In [ ]:
    !pip install spacy==2.2.3
    !python3 -m spacy download pt

In [73]:
modelo = spacy.blank('pt')
categoria = modelo.create_pipe("textcat")
categoria.add_label("ALEGRIA")
categoria.add_label("MEDO")
modelo.add_pipe(categoria)
historico = []

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.textcat.TextCategorizer object at 0x7fcccd1c8e20> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

In [72]:
modelo.begin_training()
for epoca in range(1000):
    random.shuffle(base_dados_final)
    losses = {}
    for batch in spacy.util.minibatch(base_dados_final, 30):
        textos = [modelo(texto) for texto, entities in batch]
        annotations = [{'cats': entities} for texto, entities in batch]
        modelo.update(textos, annotations, losses= losses)
    if epoca % 100 == 0:
        print(losses)
        historico.append(losses)

ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 

Etapa 6: Testes com uma frase

Etapa 7: Avaliação do Modelo